In [ ]:
!pip install bitsandbytes
!pip install {peft,trl,accelerate,datasets,transformers}

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 31.8 MB/s eta 0:00:00


In [ ]:
import os
import transformers
import torch
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer #this is used for the fine tuning
from peft import LoraConfig
from transformers import AutoTokenizer,AutoModelForCausalLM
from transformers import BitsAndBytesConfig,GemmaTokenizer

In [ ]:
os.environ["hemanth"]=userdata.get("hemanth")

In [ ]:
model_id = "google/gemma-2b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
from transformers.utils import quantization_config
tokenizer = AutoTokenizer.from_pretrained(model_id,token=os.environ["hemanth"])
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map = {"":0},
                                             token = os.environ['hemanth']
                                             )

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
text = "Quote: Imagination is more,"
device = "cuda:0"
inputs = tokenizer(text,return_tensors="pt").to(device)
outputs = model.generate(**inputs,max_new_tokens=20)
print(tokenizer.decode(outputs[0],skip_special_tokens=True))

Quote: Imagination is more, than knowledge.

I am a self-taught artist, born in 1985 in


In [ ]:
os.environ["WAND_DISABLED"]="false"

In [ ]:
lora_config = LoraConfig(
    r=8,
    target_modules = ["q_proj","o_proj","k_proj","v_proj",
                      "gate_proj","up_proj","down_proj"],
    task_type= "CAUSAL_LM"
)

In [ ]:
data = load_dataset("Abirate/english_quotes")
data = data.map(lambda samples: tokenizer(samples["quote"]),batched=True)

README.md: 0.00B [00:00, ?B/s]

quotes.jsonl:   0%|          | 0.00/647k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2508 [00:00<?, ? examples/s]

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [ ]:
data['train']['quote']

Column(['“Be yourself; everyone else is already taken.”', "“I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.”", "“Two things are infinite: the universe and human stupidity; and I'm not sure about the universe.”", '“So many books, so little time.”', '“A room without books is like a body without a soul.”'])

In [ ]:
#this is the output format
def formatting_func(example):
  text = f"Quote:{example['quote'][0]}\nAuthor: {example['author'][0]}"
  return [text]

In [ ]:
trainer = SFTTrainer(
    model= model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps = 100,
        learning_rate=2e-4,
        # fp16=True,
        bf16=True,
        logging_steps=1,
        output_dir = "outputs",
        optim="paged_adamw_8bit"),
    peft_config=lora_config,
    formatting_func=formatting_func,
)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [ ]:
trainer.train()

Step,Training Loss
1,2.561400
2,1.628800
3,2.483400
4,2.750000
5,2.297800
6,2.476700
7,2.884200
8,2.246200
9,3.186300
10,2.215100


wandb: WARNING URL not available in offline run
/usr/local/lib/python3.12/dist-packages/peft/utils/other.py:1394: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-694a7858-1dc892f542f41c9b552df73a;735593df-c828-43f6-8bca-0e52ebd4a1ba)

Cannot access gated repo for url https://huggingface.co/google/gemma-2b/resolve/main/config.json.
Access to model google/gemma-2b is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in google/gemma-2b.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:295: UserWarning: Could not find a config file in google/gemma-2b - will assume that the vocabulary was not modified.
  warnings.warn(


TrainOutput(global_step=100, training_loss=2.0594460487365724, metrics={'train_runtime': 365.7203, 'train_samples_per_second': 1.094, 'train_steps_per_second': 0.273, 'total_flos': 189744345784320.0, 'train_loss': 2.0594460487365724, 'epoch': 0.1594896331738437})

In [ ]:
text = " Quote: If you want to know what a man's like"
device = "cuda:0"
inputs = tokenizer(text,return_tensors="pt").to(device)
outputs = model.generate(**inputs,max_new_tokens=20)

print(tokenizer.decode(outputs[0],skip_special_tokens=True))

 Quote: If you want to know what a man's like, take a look at how he treats his inferiors, for that is what he will treat his
